#Search
This is the function that will return items that meet the requirements the user provided.

## Multiple Vector Search with ranking.

Ok, how to surface the best matches? We have simplified the process of outlining requirements, by separating out items we want to look for. However, now we need to combine those outputs into one single listing. 

## Reciprocal Ranked Fusion (RRF)

We perform a search across every keyword item. Then, we give each result a score based on the number of times it appears in the results. 

Some keywords will be marked 'absolute' - meaning that not being present in that keyword's list drops the item from the results entirely. 

In [54]:
#| export
from hn_jobs_chat.keys import keys

def format_job_result(job_listing):
    job = {}
    
    for key in keys:
        if key in job_listing:
            job[key] = job_listing[key]

    return job

In [65]:
#| export 
def format_results(results):
    formatted_results = []

    for job_listing in results:
        job = format_job_result(job_listing[0])

        formatted_results.append(job)

    return formatted_results


In [66]:

results = [[{ 'job_title': 'Software Engineer', 'company': 'Google', 'location': 'Mountain View, CA' }]]

format_results(results)

[{'company': 'Google',
  'location': 'Mountain View, CA',
  'job_title': 'Software Engineer'}]

In [67]:
thing = ({ 'title': 'Software Engineer', 'company': 'Google', 'location': 'Mountain View, CA' },)

type(thing[0])

dict

In [68]:
#| default_exp search
#| export
import psycopg2

month = "july"
postsTableName = "posts_" + month

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

def search(requirements):
    initial_results = {}

    for key in requirements.keys():
        query = f"SELECT row_to_json({postsTableName}) FROM {postsTableName} WHERE {key} = '{requirements[key]}' LIMIT 40;"

        cursor.execute(query)

        results = cursor.fetchall()

        formatted_results = format_results(results)

        initial_results[key] = formatted_results
        
        conn.commit()

    return initial_results

search({
    "location": "San Francisco"
})


SELECT row_to_json(posts_july) FROM posts_july WHERE location = 'San Francisco' LIMIT 10;


{'location': [{'company': 'Shortwave',
   'location': 'San Francisco',
   'timezone': None,
   'industry': None,
   'target_market': None,
   'company_status': None,
   'company_description': 'Shortwave is building an email client with a built-in AI executive assistant.',
   'company_goal': 'bringing the power of LLMs & a modern messaging UX to 4.5 billion email users.',
   'company_stage': None,
   'more_company_info': None,
   'employment_type': 'Full-time',
   'remote_or_local_details': 'ONSITE',
   'job_title': 'Sr. Software Engineer',
   'job_description': 'Help build the future by developing a modern email client with AI capabilities.',
   'job_requirements': 'Strong CS fundamentals,App development experience,Generalist background,Scrappiness,Speed,Grit',
   'job_soft_skills': None,
   'product_description': "An AI virtual assistant that has perfect knowledge of every message, attachment, receipt, newsletter, etc you've ever sent or received.",
   'tech_stack': None,
   'applicat

In [71]:
#| hide
from nbdev import nbdev_export
nbdev_export()